## TOP 10,000 랭커 유저가 사용한 선수의 20경기

TOP 10,000 랭커 유저가 사용한 선수의 20경기 평균 스탯을 조회합니다.
선수의 고유 식별자와 포지션의 목록으로 랭커 유저가 사용한 선수의 평균 스탯 기록을 조회할 수 있습니다.
한번에 너무 많은 선수목록을 입력할 경우 413 Request Entity Too Large 에러가 반환될 수 있습니다.
한번에 호출하는 선수의 수는 50명이 적당합니다.
조회하고자 하는 선수 목록을 다음과 같이 구성하여 {players} 파라미터로 전송하면 API를 조회할 수 있습니다 :

{players} : "id", "po"필드를 가지고 있는 Json Object Array
- id : 선수 고유 식별자 (spid, /metadata/spid API 참고)
- po : 선수 포지션 (spposition, /metadata/spposition API 참고)

ex) [{"id":101001183,"po":7}, {"id":214003647,"po":25},…]

*데이터는 매일 3, 9, 15, 21시에 갱신을 시작하며 최대 한시간까지 소요될 수 있습니다.
갱신된 데이터는 갱신시작 시점을 기준으로 3시간 전 데이터까지 반영됩니다.

> https://api.nexon.co.kr/fifaonline4/v1.0/rankers/status?matchtype={matchtype}&players={players}

## 패키지 설치

In [ ]:
import json
import pandas as pd
import pickle
import requests

from pandas import DataFrame
from tqdm import tqdm_notebook

In [ ]:
api_key = 'my_api_key'

API Key는 넥슨 ID로 로그인한 뒤, [넥슨 개발자센터](https://developers.nexon.com/)를 통해 발급받을 수 있습니다.

## 매치 종류

매치 종류의 메타데이터를 조회합니다.

In [ ]:
match_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/matchtype.json')
match_parsed_data = match_url.json()
match_type = pd.DataFrame(match_parsed_data)

In [ ]:
match_type.head()

매치 종류는 공식경기(50)와 감독모드(52)가 있습니다. 플레이어가 선수를 기용하여 플레이하는 스타일을 분석하기 위해 공식경기 결과만 분석에 사용합니다.

## 선수 고유 식별자

선수 고유 식별자의 메타데이터를 조회합니다. 선수 고유 식별자는 시즌아이디(seasonid) 3자리와 선수아이디(pid) 6자리로 구성되어 있습니다.

In [ ]:
spId_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spid.json')
spId_parsed_data = spId_url.json()
spId = pd.DataFrame(spId_parsed_data)

In [ ]:
spId.head()

## 선수 포지션

선수 포지션의 메타데이터를 조회합니다.

In [ ]:
spposition_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spposition.json')
spposition_parsed_data = spposition_url.json()
spposition = pd.DataFrame(spposition_parsed_data)

In [ ]:
spposition

## 조회할 선수 목록 만들기

```{players}``` 파라미터로 전송할 선수의 목록을 Json Object Array로 만듭니다. 파이썬 환경에서 구현하기 위해 리스트가 딕셔너리 형태의 데이터를 가질 수 있도록 합니다.

In [ ]:
position = list(spposition['spposition'])
player = list(spId['id'])

list_with_dic = [{"id": x, "po":y} for x in player for y in position]
list_with_dic[0:28+1]

한 선수 당 플레이 가능한 포지션은 ```0```(GK, 골키퍼)부터 ```28```(SUB, 후보선수)까지 있습니다. 후보 선수를 제외한 총 28개의 포지션을 분석합니다

## API 호출하기

In [ ]:
headers = {'Authorization' : api_key}
ranker = pd.DataFrame(columns=['createDate', 'spId', 'spPosition', 'status'])

term = 28*6 # number of data to request at once

try :
    for i in tqdm_notebook(range(0, len(list_with_dic), term)):
        r = requests.get("https://api.nexon.co.kr/fifaonline4/v1.0/rankers/status?matchtype=50&players="+str(list_with_dic[i:i+term]), headers = headers)
        # print("term: {}, get: {}".format(term, len(pd.DataFrame(r.json()))))
        if r.status_code == 404 :# Not found
            continue
        ranker = ranker.append(pd.DataFrame(r.json()))

except Exception as e :
    print("Error message: ", e)
    

ranker.reset_index(inplace = True) # Reset row num

```term```은 한번에 호출할 데이터의 개수를 의미합니다. 포지션 28개가 한 묶음으로, ```*```를 사용하여 호출할 선수의 수를 지정합니다. 예를 들어 ```term=28*3```이라면 ```[선수 A의 포지션 28개, 선수 B의 포지션 28개]```를 한번에 호출합니다.

배수가 높을수록 한번에 많은 데이터를 호출할 수 있지만, 한번에 8명 이상의 선수 조합을 호출하면 413 Request Entity Too Large 오류가 발생합니다.

## 데이터 가공하기

In [ ]:
ranker.head()

```status``` 열이 dictionary로 되어 있습니다. 열 자체를 dictionary로 변환하고 전치해서 데이터프레임에 합칩니다.

In [ ]:
rankerNew = ranker.join(pd.DataFrame(ranker["status"].to_dict()).T)
rankerNew.drop(['index', 'status'], axis=1, inplace = True)

In [ ]:
rankerNew.head()

평균 패스 성공률을 나타내는 ```passRate```를 만듭니다.

In [ ]:
rankerNew['passRate'] = rankerNew['passSuccess']/rankerNew['passTry']

In [ ]:
rankerNew.head()

분석에 사용하지 않을 변수들을 미리 제거합니다.

In [ ]:
del rankerNew['createDate']
del rankerNew['shoot']
del rankerNew['passSuccess']
del rankerNew['passTry']

메모리 사용량 최적화를 위해 특정 컬럼의 type를 지정해 줍니다.

In [ ]:
type_dic = {
    'spId': 'int32',
    'spPosition': 'int32',
    'matchCount': 'int32'
}
rankerNew = rankerNew.astype(type_dic)

rankerNew.info()

In [ ]:
rankerNew.head()

## 저장

In [ ]:
# save
with open('data.pickle', 'wb') as f:
    pickle.dump(rankerNew, f, pickle.HIGHEST_PROTOCOL)

# load
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

데이터의 크기와 메모리 사용량을 줄이기 위해 ```pickle``` 포맷으로 저장합니다.

### reference
- http://hleecaster.com/python-how-to-split-a-list-into-chunks/
- https://programmers.co.kr/learn/courses/4008/lessons/12738     
- https://specialmylife.tistory.com/entry/pandas-DataFrame-%EC%82%AC%EC%9A%A9%EB%B2%95-%EC%A0%95%EB%A6%AC%EC%9E%91%EC%97%85%EC%A4%91
- https://www.youtube.com/watch?v=0Vm9Yi_ig58&t=873s

### source
https://developers.nexon.com/fifaonline4